# NBA Statistics Predictor (2)
<span style="color: red;"><em>Do not run this is the file "player_stats.csv exists".</em></span>

## Scraping the each players stats
### From the list of names we will get each players stats

In [ ]:
# Install dependencies
%pip install string
%pip install pandas
%pip install -U scikit-learn
%pip install lxml
%pip install html5lib
import os
import pickle
from sklearn.preprocessing import LabelEncoder
import string
import pandas as pd
import random
import time
# SET THIS TO THE YEAR TO TRAIN THE MODEL FROM
year_from = 2023

In [ ]:
# import the list of players
player_list = pd.read_csv('players_list.csv')

num_players = len(player_list)


## Individual Player Data

Now there is a csv file in the directory with a list of players that retired after the year selected <br>
<br>
We will now itterate over each player and create a data frame with their game logs for their carrer <br>
The goal is to create a csv file for each all players and the games that they played <br><br>

### This block will contain helper functions for processing each player's data


In [ ]:
# returns url that contains data about what years the player played
def get_gamelog_year_url(row, year):
    player_name = row['Player'].lower().split()
    
    first_letter_last = player_name[1][0]
    unique_id = player_name[1][:5] + player_name[0][:2]
    return f"https://www.basketball-reference.com/players/{first_letter_last}/{unique_id}01/gamelog/{year}"

# returns an array of the years that the player played in the NBA
def get_player_years_played(player_row):
    return list(range(player_row["From"], player_row["To"] + 1))


def storePlayerYearData(row, year_played):
    url = get_gamelog_year_url(row, year_played)
    time.sleep(random.randint(2, 5))
    print(url)
    tables = pd.read_html(url)
    player_year_data = tables[7]
    if player_year_data is not None and not player_year_data.empty: # this could be undefined, might have played in the g-league
        player_year_data["Name"] = row["Player"]

        if os.path.exists('player_stats.csv'):
            player_stats = pd.read_csv('player_stats.csv')
            player_stats = pd.concat([player_stats, player_year_data], ignore_index=True)
            player_stats.to_csv('player_stats.csv', index=False)
        else:
            player_year_data.to_csv('player_stats.csv', index=False)

        

In [ ]:
print(f"staring to proccess player data for {num_players} players")

# itterates over every player
for index in range(3): # change this for production
    print(f"Player {index} of {num_players} players")
    # this is the player
    row = player_list.iloc[index]
    years_played = get_player_years_played(row)
    for year_played in years_played:
        storePlayerYearData(row, year_played)